Anggara Bayu P 175410142

### Loading an example dataset

scikit-learn dilengkapi dengan beberapa dataset standar, misalnya set data iris dan digit untuk klasifikasi dan dataset diabetes untuk regresi.

In [2]:
from sklearn import datasets
iris = datasets.load_iris()
digits = datasets.load_digits()

Dataset adalah objek seperti kamus yang menampung semua data dan beberapa metadata tentang data. Data ini disimpan di anggota .data , yang merupakan array.

Misalnya, dalam kasus dataset digit, digits.data memberikan akses ke fitur yang dapat digunakan untuk mengklasifikasikan sampel digit :

In [3]:
print(digits.data)

[[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]


dan digits.target memberikan kebenaran dasar untuk dataset digit, yaitu angka yang sesuai dengan setiap gambar digit.

In [4]:
digits.target

array([0, 1, 2, ..., 8, 9, 8])

#### Shape of the data arrays

Data selalu berupa array 2D, bentuk (n_samples, n_features)(8, 8), meskipun data asli mungkin memiliki bentuk yang berbeda.

In [5]:
digits.images[0]

array([[ 0.,  0.,  5., 13.,  9.,  1.,  0.,  0.],
       [ 0.,  0., 13., 15., 10., 15.,  5.,  0.],
       [ 0.,  3., 15.,  2.,  0., 11.,  8.,  0.],
       [ 0.,  4., 12.,  0.,  0.,  8.,  8.,  0.],
       [ 0.,  5.,  8.,  0.,  0.,  9.,  8.,  0.],
       [ 0.,  4., 11.,  0.,  1., 12.,  7.,  0.],
       [ 0.,  2., 14.,  5., 10., 12.,  0.,  0.],
       [ 0.,  0.,  6., 13., 10.,  0.,  0.,  0.]])

### Learning and predicting

In [6]:
from sklearn import svm
clf = svm.SVC(gamma=0.001, C=100.)

Estimator clf (untuk pengklasifikasi) pertama kali dipasang pada model; yaitu, ia harus belajar dari model. Ini dilakukan dengan melewati set pelatihan kami ke metode fit. 

In [7]:
clf.fit(digits.data[:-1], digits.target[:-1])

SVC(C=100.0, gamma=0.001)

Sekarang kita dapat memprediksi nilai baru. Dalam hal ini, kita akan memprediksi menggunakan gambar terakhir dari `digits.data`. Dengan memprediksi, kita akan menentukan gambar dari set pelatihan yang paling cocok dengan gambar terakhir.

In [8]:
clf.predict(digits.data[-1:])

array([8])

### Model persistence

In [9]:
from sklearn import svm
from sklearn import datasets
clf = svm.SVC()
X, y = datasets.load_iris(return_X_y=True)
clf.fit(X, y)

SVC()

In [10]:
import pickle
s = pickle.dumps(clf)
clf2 = pickle.loads(s)
clf2.predict(X[0:1])

array([0])

In [11]:
y[0]

0

Dalam kasus khusus scikit-learn, mungkin lebih menarik untuk menggunakan pengganti joblib untuk pickle (`joblib.dump` & `joblib.load`), yang lebih efisien pada data besar tetapi hanya bisa pickle ke disk dan bukan ke string :

In [12]:
from joblib import dump, load
dump(clf, 'filename.joblib') 

['filename.joblib']

Kemudian, kita dapat memuat ulang model pickle dengan :

In [13]:
clf = load('filename.joblib') 

### Conventions

Scikit-learn estimators mengikuti aturan tertentu untuk membuat perilakunya lebih prediktif.

#### Type casting

In [14]:
import numpy as np
from sklearn import random_projection

rng = np.random.RandomState(0)
X = rng.rand(10, 2000)
X = np.array(X, dtype='float32')
X.dtype

dtype('float32')

In [15]:
transformer = random_projection.GaussianRandomProjection()
X_new = transformer.fit_transform(X)
X_new.dtype

dtype('float64')

Dalam contoh ini, X adalah float32, yang dilemparkan float64 oleh fit_transform(X).

Target regresi dilemparkan ke float64 dan target klasifikasi dipertahankan :

In [16]:
from sklearn import datasets
from sklearn.svm import SVC
iris = datasets.load_iris()
clf = SVC()
clf.fit(iris.data, iris.target)

SVC()

In [17]:
list(clf.predict(iris.data[:3]))

[0, 0, 0]

In [18]:
clf.fit(iris.data, iris.target_names[iris.target])

SVC()

In [19]:
list(clf.predict(iris.data[:3]))

['setosa', 'setosa', 'setosa']

Di sini, yang pertama predict() mengembalikan array integer, karena iris.target (array integer) digunakan di fit. Yang kedua predict() mengembalikan array string, karena iris.target_names sudah pas.

#### Refitting and updating parameters

Parameter hiper dari estimator dapat diperbarui setelah dibangun melalui metode set_params(). Memanggil fit() lebih dari satu kali akan menimpa apa yang dipelajari oleh yang sebelumnya fit() :

In [20]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.svm import SVC
X, y = load_iris(return_X_y=True)

clf = SVC()
clf.set_params(kernel='linear').fit(X, y)

SVC(kernel='linear')

In [21]:
clf.predict(X[:5])

array([0, 0, 0, 0, 0])

In [22]:
clf.set_params(kernel='rbf').fit(X, y)

SVC()

In [23]:
clf.predict(X[:5])

array([0, 0, 0, 0, 0])

Di sini, kernel default rbf pertama diubah menjadi linear via SVC.set_params() setelah estimator dibuat, dan diubah kembali rbf untuk mereparasi estimator dan membuat prediksi kedua.

#### Multiclass vs. multilabel fitting

In [24]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelBinarizer

X = [[1, 2], [2, 4], [4, 5], [3, 2], [3, 1]]
y = [0, 0, 1, 1, 2]

classif = OneVsRestClassifier(estimator=SVC(random_state=0))
classif.fit(X, y).predict(X)

array([0, 0, 1, 1, 2])

Dalam kasus di atas, classifier cocok pada array 1d label multiclass dan predict() karena itu metode menyediakan prediksi multiclass yang sesuai. Dimungkinkan juga untuk memuat pada array 2d dari indikator label biner :

In [25]:
y = LabelBinarizer().fit_transform(y)
classif.fit(X, y).predict(X)

array([[1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 0, 0]])

Di sini, classifier fit() berada pada representasi label biner 2d y, menggunakan LabelBinarizer. Dalam hal ini predict() mengembalikan array 2d yang mewakili prediksi multilabel yang sesuai.

In [26]:
from sklearn.preprocessing import MultiLabelBinarizer
y = [[0, 1], [0, 2], [1, 3], [0, 2, 3], [2, 4]]
y = MultiLabelBinarizer().fit_transform(y)
classif.fit(X, y).predict(X)

array([[1, 1, 0, 0, 0],
       [1, 0, 1, 0, 0],
       [0, 1, 0, 1, 0],
       [1, 0, 1, 0, 0],
       [1, 0, 1, 0, 0]])